In [1]:
import random
import numpy as np
import tensorflow as tf
import random
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM


In [2]:
filepath = tf.keras.utils.get_file('shakespeare.txt',
        'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(filepath, 'rb').read().decode(encoding='utf-8')

In [3]:
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()


In [4]:
text = text[300000:800000]


In [5]:
characters = sorted(set(text))

char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

In [6]:
SEQ_LENGTH = 40
STEP_SIZE = 3

sentences = []
next_char = []


In [7]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_char.append(text[i + SEQ_LENGTH])

In [8]:
x = np.zeros((len(sentences), SEQ_LENGTH,
              len(characters)), dtype=np.bool)
y = np.zeros((len(sentences),
              len(characters)), dtype=np.bool)

for i, satz in enumerate(sentences):
    for t, char in enumerate(satz):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_char[i]]] = 1

<ipython-input-8-5d518b9a969b>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  len(characters)), dtype=np.bool)
<ipython-input-8-5d518b9a969b>:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  len(characters)), dtype=np.bool)


In [9]:
model = Sequential()
model.add(LSTM(128,
               input_shape=(SEQ_LENGTH,
                            len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.01))

model.fit(x, y, batch_size=256, epochs=20)


Epoch 1/20
651/651 [==============================] - 173s 247ms/step - loss: 2.1683
Epoch 2/20
651/651 [==============================] - 182s 279ms/step - loss: 1.7320
Epoch 3/20
651/651 [==============================] - 217s 333ms/step - loss: 1.6024
Epoch 4/20
651/651 [==============================] - 182s 279ms/step - loss: 1.5331
Epoch 5/20
651/651 [==============================] - 189s 290ms/step - loss: 1.4847
Epoch 6/20
651/651 [==============================] - 142s 218ms/step - loss: 1.4494
Epoch 7/20
651/651 [==============================] - 110s 170ms/step - loss: 1.4209
Epoch 8/20
651/651 [==============================] - 110s 169ms/step - loss: 1.3976
Epoch 9/20
651/651 [==============================] - 113s 173ms/step - loss: 1.3780
Epoch 10/20
651/651 [==============================] - 111s 170ms/step - loss: 1.3634
Epoch 11/20
651/651 [==============================] - 111s 171ms/step - loss: 1.3492
Epoch 12/20
651/651 [==============================] - 111s 171

In [11]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [12]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1

        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions,
                                 temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

In [13]:
print(generate_text(300, 0.2))
print(generate_text(300, 0.4))
print(generate_text(300, 0.5))
print(generate_text(300, 0.6))
print(generate_text(300, 0.7))
print(generate_text(300, 0.8))

et me have
a dram of poison, such soon-sleast of the ways
should stay the countract the complio;
and the truth, that is the straight is the charrents of the bloody
of the charrom of the charrents the countract
the truth and the charrents the bark of the charrents
the constines of the brother the breath the contrall
and the complio the str
name in england;
and i must find that time is not the court.

mercutio:
good my be repart the servand of the cast
a mortal tonguess the world is the bark.

king richard ii:
and be thy speak of the charrents of their heads,
and i have spent to be a princh and the foul death,
and this thou slain thy hand of england,
she chaige his part the 
request, to stand aside,
while i use furch the brother and all the streas.

post:
ay, i have ster the crown of this will be this kins
and be abless of this is sunserfecty here.

romeo:
and thou hast the be not to my son and they see
the crowner therefore and the world and will
to sing of the strength hours hence be m